## Imports

In [1]:
import pandas as pd
import numpy as np
import re
import os
import folium

## Data Read-in

In [2]:
df = pd.read_csv('redfin_2023-06-02-11-12-30.csv')

## Data Clean

In [3]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [4]:
df = df.dropna(subset=['SOLD DATE'])

In [5]:
# Define list of desired months (excluding current month)
desired_months = ['May']

# Filter DataFrame to include only entries from desired months
df_filtered = df[df['SOLD DATE'].str.split('-', expand=True)[0].isin(desired_months)]

# Reset the index
df_filtered = df_filtered.reset_index(drop=True)

In [6]:
# Data checks
print(df_filtered['PRICE'].isna().value_counts())
print('-------')
print(df_filtered['$/SQUARE FEET'].isna().value_counts())
print('-------')
print(df_filtered['YEAR BUILT'].isna().value_counts())
print('-------')

PRICE
False    985
Name: count, dtype: int64
-------
$/SQUARE FEET
False    974
True      11
Name: count, dtype: int64
-------
YEAR BUILT
False    985
Name: count, dtype: int64
-------


In [7]:
df_filtered.loc[df_filtered['PRICE'] == '0']

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE


In [8]:
df_filtered['PRICE'] = pd.to_numeric(df_filtered['PRICE'])
df_filtered['$/SQUARE FEET'] = pd.to_numeric(df_filtered['$/SQUARE FEET'])
df_filtered['YEAR BUILT'] = pd.to_numeric(df_filtered['YEAR BUILT'])
df_filtered['LATITUDE'] = pd.to_numeric(df_filtered['LATITUDE'])
df_filtered['LONGITUDE'] = pd.to_numeric(df_filtered['LONGITUDE'])

In [9]:
df_filtered.sort_values(by='PRICE',ascending=True).head(20)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
22,PAST SALE,May-2-2023,Condo/Co-op,1800 Sans Souci Blvd #309,North Miami,FL,33181.0,180.0,1.0,1.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/North-Miami/1800-San...,MARMLS,A11365579,N,Y,25.885487,-80.161234
219,PAST SALE,May-17-2023,Condo/Co-op,10 Aragon Ave #918,Coral Gables,FL,33134.0,470.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Coral-Gables/10-Arag...,MARMLS,A11366777,N,Y,25.750306,-80.256031
266,PAST SALE,May-17-2023,Condo/Co-op,19080 NE 3rd Ct #309,Miami,FL,33179.0,65000.0,0.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/19080-NE-3rd-C...,MARMLS,A11369815,N,Y,25.949846,-80.194561
129,PAST SALE,May-5-2023,Condo/Co-op,65 NE 202nd Ter Unit Q28,Miami Gardens,FL,33179.0,87500.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Gardens/65-NE-...,MARMLS,A11361957,N,Y,25.960422,-80.200578
154,PAST SALE,May-5-2023,Condo/Co-op,270 NE 191st St #105,Miami,FL,33179.0,90000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/270-NE-191st-S...,MARMLS,A11360799,N,Y,25.949925,-80.195745
277,PAST SALE,May-22-2023,Condo/Co-op,18900 NE 3rd Ct #510,Miami,FL,33179.0,90000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/18900-NE-3rd-C...,MARMLS,A11352705,N,Y,25.948300,-80.194539
789,PAST SALE,May-31-2023,Condo/Co-op,19051 NE 2nd Ave #1515,Miami,FL,33179.0,92000.0,2.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/19051-NE-2nd-A...,MARMLS,A11325048,N,Y,25.949735,-80.197046
106,PAST SALE,May-16-2023,Condo/Co-op,20210 NE 2nd Ave Unit V31,Miami Gardens,FL,33179.0,95000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Gardens/20210-...,MARMLS,A11371220,N,Y,25.958779,-80.198967
748,PAST SALE,May-23-2023,Condo/Co-op,11000 SW 200th St #409,Miami,FL,33157.0,100000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Cutler-Bay/11000-SW-...,MARMLS,A11300236,N,Y,25.580326,-80.370222
534,PAST SALE,May-26-2023,Condo/Co-op,1475 NE 125th Ter Unit 612A,North Miami,FL,33161.0,105000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/North-Miami/1475-NE-...,MARMLS,A11352944,N,Y,25.892005,-80.168544


In [11]:
print(df_filtered['URL'].iloc[219])

https://www.redfin.com/FL/Coral-Gables/10-Aragon-Ave-33134/unit-918/home/43331737


In [12]:
# Correct the prices, if needed
df_filtered.at[22,'PRICE']=(180000)
df_filtered.at[219,'PRICE']=(470000)

In [13]:
# Find problem psf by searching for a '0' value
df_filtered.loc[df_filtered['$/SQUARE FEET'] == '0'][['SOLD DATE','ADDRESS','CITY','$/SQUARE FEET','PRICE','SQUARE FEET']]

,SOLD DATE,ADDRESS,CITY,$/SQUARE FEET,PRICE,SQUARE FEET


In [15]:
# # Corrections, if needed
df_filtered.at[22,'$/SQUARE FEET']=(180000/714)
df_filtered.at[219,'$/SQUARE FEET']=(470000/798)

In [16]:
# Find problem psf by searching for low values
df_filtered.sort_values(by='$/SQUARE FEET',ascending=True).head(20)[['PRICE','ADDRESS','CITY','$/SQUARE FEET']]

,PRICE,ADDRESS,CITY,$/SQUARE FEET
789,92000.0,19051 NE 2nd Ave #1515,Miami,90.0
129,87500.0,65 NE 202nd Ter Unit Q28,Miami Gardens,105.0
277,90000.0,18900 NE 3rd Ct #510,Miami,106.0
106,95000.0,20210 NE 2nd Ave Unit V31,Miami Gardens,114.0
259,150000.0,1670 NE 191st St Unit 211-3,Miami,114.0
348,155000.0,496 NW 165th St Rd Unit D412,Miami,132.0
534,105000.0,1475 NE 125th Ter Unit 612A,North Miami,136.0
716,130000.0,1710 NE 191st St Unit 307-3,Miami,143.0
904,155000.0,5300 NW 87th Ave #1116,Doral,145.0
154,90000.0,270 NE 191st St #105,Miami,150.0


In [17]:
print(df_filtered.URL.iloc[789])

https://www.redfin.com/FL/Miami/19051-NE-2nd-Ave-33179/unit-1515/home/43004039


In [17]:
# # Drop sales that aren't condos but labeled as such
# df_filtered = df_filtered.drop(1320)

## Make Maps

In [18]:
### Create a price column formatted as currency ###
df_filtered['PRICE_AS_CURRENCY'] = df_filtered['PRICE'].apply(lambda x: "${:,.0f}".format(x))
### Set formatting for Beds, Baths ###
df_filtered['YEAR BUILT DISPLAY'] = df_filtered['YEAR BUILT'].apply(lambda x: '{:.0f}'.format(x))
df_filtered['PRICE_SQUARE_FEET_AS_CURRENCY'] = df_filtered['$/SQUARE FEET'].apply(lambda x: '${:,.0f}'.format(x))

In [19]:
df_filtered = df_filtered.sort_values(by=['PRICE'], ascending=False)
### Insert different colors for top 10 sales vs. the rest ###
df_filtered['COLOR'] = ''
### Create RANK column ###
df_filtered['RANK'] = 0
### Insert RANK values ###
df_filtered['RANK'] = range(1, len(df_filtered) + 1)
# use numpy to assign values to the 'COLOR' column
df_filtered['COLOR'] = np.where(df_filtered['RANK'] <= 10, 'orange', 'blue')

## HTML Popup Formatter

In [20]:
### Define list of columns to drop from DF ###
columns_drop = ['SALE TYPE','PROPERTY TYPE','STATE OR PROVINCE','ZIP OR POSTAL CODE','HOA/MONTH','STATUS','NEXT OPEN HOUSE START TIME','NEXT OPEN HOUSE END TIME','SOURCE','MLS#','FAVORITE','INTERESTED','SQUARE FEET','LOT SIZE']

In [21]:
### Drop the columns ###
df_filtered = df_filtered.drop(columns=columns_drop)

In [22]:
def popup_html(row):
    Price = row['PRICE_AS_CURRENCY']
    Address = row['ADDRESS']
    City = row['CITY']
    sold_date = row['SOLD DATE']
    beds = row['BEDS']
    baths = row['BATHS']
    psf = row['PRICE_SQUARE_FEET_AS_CURRENCY']
    year_built = row['YEAR BUILT DISPLAY']
    rank = row['RANK']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Price: </strong>{}'''.format(Price) + '''<br>
    <strong>Address: </strong>{}'''.format(Address) + '''<br>
    <strong>City: </strong>{}'''.format(City) + '''<br>
    <strong>Sold: </strong>{}'''.format(sold_date) + '''<br>
    <strong>Beds: </strong>{}'''.format(beds) + '''<br>
    <strong>Baths: </strong>{}'''.format(baths) + '''<br>
    <strong>Price per sf: </strong>{}'''.format(psf) + '''<br>
    <strong>Year Built: </strong>{}'''.format(year_built) + '''<br>
    <strong>Price Rank: </strong>{}'''.format(rank) + '''
    </html>
    '''
    return html

In [23]:
### Create map container ###
m = folium.Map(location=df_filtered[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=10,tiles=None)

### Create title ###
title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f"May 2023 Condo Sales")

m.get_root().html.add_child(folium.Element(title_html))

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top 10 Sales')

for index, row in df_filtered.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)

# Display map

In [24]:
m.save('index.html')

## Summary Info

In [25]:
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

Newest = '\033[1m' + 'Newest' + '\033[0m'
Oldest = '\033[1m' + 'Oldest' + '\033[0m'

In [26]:
df_filtered.columns

Index(['SOLD DATE', 'ADDRESS', 'CITY', 'PRICE', 'BEDS', 'BATHS', 'LOCATION',
       'YEAR BUILT', 'DAYS ON MARKET', '$/SQUARE FEET', 'URL', 'LATITUDE',
       'LONGITUDE', 'PRICE_AS_CURRENCY', 'YEAR BUILT DISPLAY',
       'PRICE_SQUARE_FEET_AS_CURRENCY', 'COLOR', 'RANK'],
      dtype='object')

In [27]:
df_filtered['FULL_ADDRESS'] = df_filtered['ADDRESS'] + ' ' + df_filtered['CITY']

In [28]:
print(df_filtered.loc[df_filtered['PRICE'].idxmin()]['URL'])

https://www.redfin.com/FL/Miami/19080-NE-3rd-Ct-33179/unit-309/home/43012055


In [29]:
print(f"{ME}{BR}{df_filtered.loc[df_filtered['PRICE'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['PRICE'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['PRICE'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['PRICE'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['PRICE'].idxmax()]['YEAR BUILT']:.0f}")
print(f"{LE}{BR}{df_filtered.loc[df_filtered['PRICE'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['PRICE'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['PRICE'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['PRICE'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['PRICE'].idxmin()]['YEAR BUILT']:.0f}")

print(f"{MAX_PSF}{BR}{df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['YEAR BUILT']:.0f}")
print(f"{MIN_PSF}{BR}{df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['YEAR BUILT']:.0f}")

print(f"{Newest}{BR}{df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['YEAR BUILT']:.0f}")
print(f"{Oldest}{BR}{df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['YEAR BUILT']:.0f}")

Most Expensive
OCEANA BAL HARBOUR CONDO, 10201 Collins Ave #2803 Bal Harbour | Price $22,000,000 | $4,245 psf | Year built: 2017
Least Expensive
STAR LAKES ESTATES NO 3 C, 19080 NE 3rd Ct #309 Miami | Price $65,000 | $152 psf | Year built: 1968
Highest Price Per Square Foot
OCEANA BAL HARBOUR CONDO, 10201 Collins Ave #2803 Bal Harbour | Price $22,000,000 | $4,245 psf | Year built: 2017
Lowest Price Per Square Foot
STAR LAKE ESTATES NO 15 C, 19051 NE 2nd Ave #1515 Miami | Price $92,000 | $90 psf | Year built: 1967
Newest
ESTATES THE ACQUALINA, 17975 Collins #2701 Sunny Isles Beach | Price $10,965,000 | $2,294 psf | Year built: 2023
Oldest
Bristol Condo, 1350 Pennsylvania Ave #101 Miami Beach | Price $390,000 | $448 psf | Year built: 1925


## Time on Market Calculator

In [29]:
# print(df_Current.loc[df_Current['YEAR BUILT'].idxmin()])

In [82]:
from datetime import datetime, timedelta

date1 = datetime(2022, 2, 23) ## List (Earlier) date
date2 = datetime(2023, 4, 11) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

412


## Map URL Snagger

In [74]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

In [75]:
cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/MD_condo_sales_month_ending_april_2023


## Get Summary Data

In [76]:
print('SALES INFO')
print(f'Number of sales: {len(df_filtered)}')
print('--------')
print(f'Total sale price: ${df_filtered["PRICE"].sum():,.0f}')
print('--------')
print(f'Median sale price: ${df_filtered["PRICE"].median():,.0f}')
print('--------')
print(f'Max sale price: ${df_filtered["PRICE"].max():,.0f}')
print('--------')
print(f'Min sale price: ${df_filtered["PRICE"].min():,.0f}')
print('------------------------------------------------')
print('PSF INFO')
print(f'Max price per square foot: ${df_filtered["$/SQUARE FEET"].max():,.0f}')
print('--------')
print(f'Min price per square foot: ${df_filtered["$/SQUARE FEET"].min():,.0f}')
print('--------')
print(f'Median price per square foot: ${df_filtered["$/SQUARE FEET"].median():,.0f}')
print('------------------------------------------------')
print('CONDO AGES')
print(f'Newest building: {df_filtered["YEAR BUILT"].max()}')
print('----------')
print(f'Oldest building: {df_filtered["YEAR BUILT"].min()}')
print('----------')
print(f'Average building age: {df_filtered["YEAR BUILT"].mean()}')

SALES INFO
Number of sales: 930
--------
Total sale price: $646,429,006
--------
Median sale price: $410,000
--------
Max sale price: $17,750,000
--------
Min sale price: $85,000
------------------------------------------------
PSF INFO
Max price per square foot: $3,900
--------
Min price per square foot: $94
--------
Median price per square foot: $432
------------------------------------------------
CONDO AGES
Newest building: 2022.0
----------
Oldest building: 1925.0
----------
Average building age: 1989.1204301075268
